In [4]:
import pyspark
import requests
import hashlib

from pyspark.sql import SparkSession 

spark = SparkSession.builder \
    .master("local") \
    .appName("PPRL") \
    .getOrCreate()

request = requests.get(f'http://snf-21129.ok-kno.grnetcloud.net:9200//take_data/0')
url_content = request.content
csv_file = open("a_download.csv", 'wb')
csv_file.write(url_content)
csv_file.close()

request = requests.get(f'http://snf-21129.ok-kno.grnetcloud.net:9300//take_data/0')
url_content = request.content
csv_file = open("b_download.csv", 'wb')
csv_file.write(url_content)
csv_file.close()

df_1 = spark.read.format("csv").option("header",    "true").load("a_download.csv")
df_2 = spark.read.format("csv").option("header",    "true").load("b_download.csv")
new_df = df_1.join(df_2)

new_df.write.csv("./joined_data")
